# Modelo con Regresion Logistica 

## Importar librerías

In [268]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np 
#from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow import keras
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


## Funcion para Evaluación del modelo

In [275]:
from sklearn.metrics import mean_squared_error, r2_score

    
def evaluate_model(y_pred, y_test):
    # Calculate evaluation metrics
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Print evaluation metrics
    print("Mean Squared Error:", mse)
    print("R^2:", r2)
    

## Carga de datos

In [276]:
df = pd.read_csv('../Data Engineering/datoslimpios.csv')


## filtado de variables numericas 

In [277]:
# Eliminar la variable 'date' del DataFrame
df = df.drop(columns=['wip'])
df = df.drop(columns=['date'])

# Filtrar solo variables numéricas
df_numeric = df.select_dtypes(include='number')

# División de los datos en características (X) y etiquetas (y)
x = df_numeric.drop(columns=['actual_productivity'])  
y = df_numeric['actual_productivity']



## Split de datos en train y test

In [278]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)


In [279]:
x_train


,team,targeted_productivity,smv,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers
561,8,0.65,4.15,960,0,0.0,0,0,8.0
879,8,0.70,29.40,4560,40,0.0,0,2,58.0
847,10,0.80,3.94,2400,0,0.0,0,0,20.0
1057,11,0.80,11.41,3420,50,0.0,0,0,58.0
175,1,0.80,26.16,10620,75,0.0,0,0,59.0
...,...,...,...,...,...,...,...,...,...
1033,11,0.80,2.90,1200,0,0.0,0,0,10.0
763,11,0.70,11.41,2160,30,0.0,0,2,54.0
835,6,0.75,18.79,3960,45,0.0,0,0,33.0
559,11,0.70,20.55,6480,60,0.0,0,0,54.0


## Entrenamiento de modelo

In [280]:
x_train.dtypes

team                       int64
targeted_productivity    float64
smv                      float64
over_time                  int64
incentive                  int64
idle_time                float64
idle_men                   int64
no_of_style_change         int64
no_of_workers            float64
dtype: object

In [282]:


def build_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')  # Capa de salida para regresión logística
    ])
    return model


def train_neural_network(X_train, y_train, epochs=10, batch_size=32):
    # Normalizar características
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    
    # Construir el modelo
    input_shape = X_train_scaled.shape[1:]
    model = build_model(input_shape)
    
    # Compilar el modelo
    model.compile(optimizer='adam',
                  loss='mean_squared_error',  # Binary crossentropy para regresión logística
                  metrics=['mean_squared_error'])  # Métrica de precisión
    
    # Entrenar el modelo
    model.fit(X_train_scaled, y_train, epochs=epochs, batch_size=batch_size)

    return model


# Ejemplo de uso
# Suponiendo que tienes X_train y y_train
model = train_neural_network(x_train, y_train)

y_pred = model.predict(x_test)

# Entrenar el modelo con las variables categóricas incluidas
#model_encoded = train_neural_network(x_train, y_train)

Epoch 1/10
29/29 [==============================] - 0s 1ms/step - loss: 0.0708 - mean_squared_error: 0.0708
Epoch 2/10
29/29 [==============================] - 0s 1ms/step - loss: 0.0326 - mean_squared_error: 0.0326
Epoch 3/10
29/29 [==============================] - 0s 1ms/step - loss: 0.0248 - mean_squared_error: 0.0248
Epoch 4/10
29/29 [==============================] - 0s 1ms/step - loss: 0.0224 - mean_squared_error: 0.0224
Epoch 5/10
29/29 [==============================] - 0s 1ms/step - loss: 0.0209 - mean_squared_error: 0.0209
Epoch 6/10
29/29 [==============================] - 0s 1ms/step - loss: 0.0197 - mean_squared_error: 0.0197
Epoch 7/10
29/29 [==============================] - 0s 1ms/step - loss: 0.0190 - mean_squared_error: 0.0190
Epoch 8/10
29/29 [==============================] - 0s 979us/step - loss: 0.0185 - mean_squared_error: 0.0185
Epoch 9/10
29/29 [==============================] - 0s 1ms/step - loss: 0.0182 - mean_squared_error: 0.0182
Epoch 10/10
8/8 [=========

## Evaluar modelo 

In [283]:
evaluar = evaluate_model(y_pred, y_test)

Mean Squared Error: 0.11231295407207753
R^2: -2.8839206984822168


## Carga de datos

In [292]:
df = pd.read_csv('../Data Engineering/datoslimpios.csv')


## filtado de variables numericas y categoricas


In [293]:
# Eliminar la variable 'date' del DataFrame
df = df.drop(columns=['wip'])
df = df.drop(columns=['date'])

# Codificación one-hot de las variables categóricas
df_encoded = pd.get_dummies(df, columns=['quarter', 'department', 'day'])

# División de los datos en características (X) y etiquetas (y)
x_encoded = df_encoded.drop(columns=['actual_productivity'])  
y_encoded = df_encoded['actual_productivity']




## Split de datos en train y test

In [294]:
# División de los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test= train_test_split(x_encoded, y_encoded, test_size=0.2, random_state=0)

In [295]:
x_train


,team,targeted_productivity,smv,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,quarter_Quarter1,...,quarter_Quarter4,quarter_Quarter5,department_finishing,department_sweing,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday
561,8,0.65,4.15,960,0,0.0,0,0,8.0,True,...,False,False,True,False,True,False,False,False,False,False
879,8,0.70,29.40,4560,40,0.0,0,2,58.0,False,...,True,False,False,True,False,False,False,False,True,False
847,10,0.80,3.94,2400,0,0.0,0,0,20.0,False,...,True,False,True,False,True,False,False,False,False,False
1057,11,0.80,11.41,3420,50,0.0,0,0,58.0,True,...,False,False,False,True,False,True,False,False,False,False
175,1,0.80,26.16,10620,75,0.0,0,0,59.0,False,...,False,False,False,True,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033,11,0.80,2.90,1200,0,0.0,0,0,10.0,True,...,False,False,True,False,False,False,False,True,False,False
763,11,0.70,11.41,2160,30,0.0,0,2,54.0,False,...,False,False,False,True,True,False,False,False,False,False
835,6,0.75,18.79,3960,45,0.0,0,0,33.0,False,...,True,False,False,True,False,False,True,False,False,False
559,11,0.70,20.55,6480,60,0.0,0,0,54.0,True,...,False,False,False,True,True,False,False,False,False,False


## Entrenamiento de modelo

In [296]:
x_train.dtypes

team                       int64
targeted_productivity    float64
smv                      float64
over_time                  int64
incentive                  int64
idle_time                float64
idle_men                   int64
no_of_style_change         int64
no_of_workers            float64
quarter_Quarter1            bool
quarter_Quarter2            bool
quarter_Quarter3            bool
quarter_Quarter4            bool
quarter_Quarter5            bool
department_finishing        bool
department_sweing           bool
day_Monday                  bool
day_Saturday                bool
day_Sunday                  bool
day_Thursday                bool
day_Tuesday                 bool
day_Wednesday               bool
dtype: object

In [298]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def build_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')  # Capa de salida para regresión logística
    ])
    return model


def train_neural_network(X_train, y_train, epochs=10, batch_size=32):
    # Normalizar características
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    
    # Construir el modelo
    input_shape = X_train_scaled.shape[1:]
    model = build_model(input_shape)
    
    # Compilar el modelo
    model.compile(optimizer='adam',
                  loss='mean_squared_error',  # Binary crossentropy para regresión logística
                  metrics=['mean_squared_error'])  # Métrica de precisión
    
    # Entrenar el modelo
    model.fit(X_train_scaled, y_train, epochs=epochs, batch_size=batch_size)

    return model


# Ejemplo de uso
# Suponiendo que tienes X_train y y_train
model = train_neural_network(x_train, y_train)

y_pred = model.predict(x_test)

# Entrenar el modelo con las variables categóricas incluidas
#model_encoded = train_neural_network(x_train, y_train)

Epoch 1/10
29/29 [==============================] - 0s 1ms/step - loss: 0.0870 - mean_squared_error: 0.0870
Epoch 2/10
29/29 [==============================] - 0s 1ms/step - loss: 0.0311 - mean_squared_error: 0.0311
Epoch 3/10
29/29 [==============================] - 0s 1ms/step - loss: 0.0232 - mean_squared_error: 0.0232
Epoch 4/10
29/29 [==============================] - 0s 1ms/step - loss: 0.0209 - mean_squared_error: 0.0209
Epoch 5/10
29/29 [==============================] - 0s 1ms/step - loss: 0.0196 - mean_squared_error: 0.0196
Epoch 6/10
29/29 [==============================] - 0s 1ms/step - loss: 0.0187 - mean_squared_error: 0.0187
Epoch 7/10
29/29 [==============================] - 0s 1ms/step - loss: 0.0181 - mean_squared_error: 0.0181
Epoch 8/10
29/29 [==============================] - 0s 1ms/step - loss: 0.0177 - mean_squared_error: 0.0177
Epoch 9/10
29/29 [==============================] - 0s 1ms/step - loss: 0.0173 - mean_squared_error: 0.0173
Epoch 10/10
29/29 [=========

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [217]:
# Convert test data to float type
x_test = x_test.astype(float)
y_test = y_test.astype(float)

# Convert test data to tensors
x_test_tensor = tf.convert_to_tensor(x_test)
y_test_tensor = tf.convert_to_tensor(y_test)

# Evaluate the model
model.evaluate(x_test_tensor, y_test_tensor)


8/8 [==============================] - 0s 1ms/step - loss: 0.1387 - mean_squared_error: 0.1387


[0.13873879611492157, 0.13873879611492157]

# finishing regresion 

NO sirven:   idle_time, idle_men, no_of_style_change_wip, incentive

SI Puede:    smv   targeted   no_workers   over time      significativas en analisis de datos 

In [263]:
df = pd.read_csv('../Data Engineering/datoslimpios.csv')

#filtrar datos de sweing
df = df[df['department'] == 'finishing']

df = df.drop(columns=['date'])



In [264]:
# Filtrar solo variables numéricas
df_numeric = df.select_dtypes(include='number')

# División de los datos en características (X) y etiquetas (y)
X = df_numeric.drop(columns=['actual_productivity'])  
y = df_numeric['actual_productivity']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [265]:

# Crear el modelo de regresión lineal
model = LinearRegression()

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

 

## Evaluar modelo 

In [266]:
evaluar = evaluate_model(y_pred, y_test)

Mean Squared Error: 0.02787743172587364
R^2: 0.005359668921921523


# finishing con red neuronal 

In [215]:
df = pd.read_csv('../Data Engineering/datoslimpios.csv')

#filtrar datos de sweing
df = df[df['department'] == 'finishing']


In [216]:
X = df[['smv', "over_time", "no_of_workers" ]]
y = df['actual_productivity']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [217]:


# Get the input shape from the training data
input_dim = X_train.shape[1]

# Create a sequential model
model = Sequential()

# Add layers to the model
model.add(Dense(64, activation='relu', input_shape=(input_dim,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Evaluate the model
y_pred = model.predict(X_test)



Epoch 1/100
13/13 [==============================] - 0s 2ms/step - loss: 0.1075 - mse: 0.1075
Epoch 2/100
13/13 [==============================] - 0s 1ms/step - loss: 0.1075 - mse: 0.1075
Epoch 3/100
13/13 [==============================] - 0s 1ms/step - loss: 0.1075 - mse: 0.1075
Epoch 4/100
13/13 [==============================] - 0s 1ms/step - loss: 0.1075 - mse: 0.1075
Epoch 5/100
13/13 [==============================] - 0s 1ms/step - loss: 0.1075 - mse: 0.1075
Epoch 6/100
13/13 [==============================] - 0s 1ms/step - loss: 0.1075 - mse: 0.1075
Epoch 7/100
13/13 [==============================] - 0s 1ms/step - loss: 0.1075 - mse: 0.1075
Epoch 8/100
13/13 [==============================] - 0s 1ms/step - loss: 0.1075 - mse: 0.1075
Epoch 9/100
13/13 [==============================] - 0s 1ms/step - loss: 0.1075 - mse: 0.1075
Epoch 10/100
13/13 [==============================] - 0s 1ms/step - loss: 0.1075 - mse: 0.1075
Epoch 11/100
13/13 [==============================] - 0s 1m

## Evaluar modelo 

In [218]:
evaluar = evaluate_model(y_pred, y_test)

Mean Squared Error: 0.07960534301299793
R^2: -1.8402431582872403


# sweing con regrecion lineal

## Modelo con solo incentive

In [184]:

df = pd.read_csv('../Data Engineering/datoslimpios.csv')

#filtrar datos de sweing
df = df[df['department'] == 'sweing']


# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='actual_productivity'), df['actual_productivity'], test_size=0.2, random_state=0)

In [185]:
from sklearn.linear_model import LinearRegression



# Crear el modelo de regresión lineal
model = LinearRegression()

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train[['incentive']], y_train)

y_pred = model.predict(X_test[['incentive']])

 

## Evaluar modelo 

In [186]:
evaluar = evaluate_model(y_pred, y_test)

Mean Squared Error: 0.007250628720285476
R^2: 0.6044576245809388


## modelo con incentive y wip

In [187]:
df = pd.read_csv('../Data Engineering/datoslimpios.csv')

#filtrar datos de sweing
df = df[df['department'] == 'sweing']


In [188]:
X = df[['incentive', "wip"]]
y = df['actual_productivity']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [189]:

# Crear el modelo de regresión lineal
model = LinearRegression()

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

 

## Evaluar modelo 

In [190]:
evaluar = evaluate_model(y_pred, y_test)

Mean Squared Error: 0.007252893611454882
R^2: 0.6043340683394356


## modelo con incentive, svm y wip

In [191]:
df = pd.read_csv('../Data Engineering/datoslimpios.csv')

#filtrar datos de sweing
df = df[df['department'] == 'sweing']



In [192]:
X = df[['incentive', "wip", "smv"]]
y = df['actual_productivity']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [193]:

# Crear el modelo de regresión lineal
model = LinearRegression()

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

 

## Evaluar modelo 

In [194]:
evaluar = evaluate_model(y_pred, y_test)

Mean Squared Error: 0.007206517653840272
R^2: 0.6068640084515111


# Sweing con red neuronal 

## Modelo con solo incentive

In [199]:
df = pd.read_csv('../Data Engineering/datoslimpios.csv')

#filtrar datos de sweing
df = df[df['department'] == 'sweing']


In [200]:
X = df[['incentive']]
y = df['actual_productivity']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [201]:


# Get the input shape from the training data
input_dim = X_train.shape[1]

# Create a sequential model
model = Sequential()

# Add layers to the model
model.add(Dense(64, activation='relu', input_shape=(input_dim,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Evaluate the model
y_pred = model.predict(X_test)



Epoch 1/100
17/17 [==============================] - 1s 1ms/step - loss: 0.3194 - mse: 0.3194
Epoch 2/100
17/17 [==============================] - 0s 1ms/step - loss: 0.0450 - mse: 0.0450
Epoch 3/100
17/17 [==============================] - 0s 2ms/step - loss: 0.0153 - mse: 0.0153
Epoch 4/100
17/17 [==============================] - 0s 2ms/step - loss: 0.0108 - mse: 0.0108
Epoch 5/100
17/17 [==============================] - 0s 1ms/step - loss: 0.0096 - mse: 0.0096
Epoch 6/100
17/17 [==============================] - 0s 1ms/step - loss: 0.0092 - mse: 0.0092
Epoch 7/100
17/17 [==============================] - 0s 2ms/step - loss: 0.0091 - mse: 0.0091
Epoch 8/100
17/17 [==============================] - 0s 1ms/step - loss: 0.0091 - mse: 0.0091
Epoch 9/100
17/17 [==============================] - 0s 1ms/step - loss: 0.0091 - mse: 0.0091
Epoch 10/100
17/17 [==============================] - 0s 1ms/step - loss: 0.0091 - mse: 0.0091
Epoch 11/100
17/17 [==============================] - 0s 2m

## Evaluar modelo 

In [202]:
evaluar = evaluate_model(y_pred, y_test)

Mean Squared Error: 0.007168977699018981
R^2: 0.6089119195328764
